In [55]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [56]:
d1 = 183.3 #joint 1 height [mm]
d2 = 737.31 #joint 2 radius [mm]
d3 = 387.8 #joint 3 radius [mm]
d4 = 95.5 #joint 4 radius [mm]
d5 = 115.5 #joint 5 radius [mm]
d6 = 76.8 #end effector offset
q = sp.pi/2 #standard 90 degree angle



In [57]:
#Initialize symbols
thetai = sp.Symbol('theta_i')

t = sp.Symbol('t')
ai = sp.Symbol('a_i')
di = sp.Symbol('d_i')
alphai = sp.Symbol('alpha_i')
pi = sp.Symbol('pi')

In [58]:
thetas = sp.symbols('theta_1:7')
dh_theta = [thetas[0], thetas[1]+q, thetas[2], thetas[3]+q, thetas[4], thetas[5]]
dh_a = [0, d2, d3, 0, 0, 0]
dh_d = [d1, 0, 0, d4, d5, d6]
dh_alpha = [q, 0, 0, q, -q, 0]

In [59]:
#Create dh table based on chosen coordinate frames

dh_table = {
    "": ["to 1", "to 2", "to 3", "to 4", "to 5", "to 6"],
    thetai: [dh_theta[0],dh_theta[1],dh_theta[2],dh_theta[3],dh_theta[4],dh_theta[5]],
    ai: [dh_a[0],dh_a[1],dh_a[2],dh_a[3],dh_a[4],dh_a[5]],
    di: [dh_d[0],dh_d[1],dh_d[2],dh_d[3],dh_d[4],dh_d[5]],
    alphai: [dh_alpha[0],dh_alpha[1],dh_alpha[2],dh_alpha[3],dh_alpha[4],dh_alpha[5]]

}

df = pd.DataFrame(dh_table)
df

,,theta_i,a_i,d_i,alpha_i
0,to 1,theta_1,0.00,183.3,pi/2
1,to 2,theta_2 + pi/2,737.31,0.0,0
2,to 3,theta_3,387.80,0.0,0
3,to 4,theta_4 + pi/2,0.00,95.5,pi/2
4,to 5,theta_5,0.00,115.5,-pi/2
5,to 6,theta_6,0.00,76.8,0


In [60]:
A_array = [sp.zeros(4,4)]*6
for i in range(0,6):
    A_array[i] = sp.Matrix([
        [sp.cos(dh_theta[i]),-sp.sin(dh_theta[i])*sp.cos(dh_alpha[i]),sp.sin(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.cos(dh_theta[i])],
        [sp.sin(dh_theta[i]),sp.cos(dh_theta[i])*sp.cos(dh_alpha[i]),-sp.cos(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.sin(dh_theta[i])],
        [0,sp.sin(dh_alpha[i]),sp.cos(dh_alpha[i]),dh_d[i]],
        [0,0,0,1]
    ])

A_final = A_array[0]
for i in range(1,6):
    A_final = A_final*A_array[i]

A_products = [A_array[0]]*6
for i in range(1,6):
    A_products[i] = A_products[i-1]*A_array[i]



In [61]:
def subs_function(matrix,sub_values):
    result = matrix.subs([
        (thetas[0],sub_values[0]),
        (thetas[1],sub_values[1]),
        (thetas[2],sub_values[2]),
        (thetas[3],sub_values[3]),
        (thetas[4],sub_values[4]),
        (thetas[5],sub_values[5])     
    ])
    return result

def subs_position_vector(matrix,sub_values):
    msubs = matrix.subs([
        (thetas[0],sub_values[0]),
        (thetas[1],sub_values[1]),
        (thetas[2],sub_values[2]),
        (thetas[3],sub_values[3]),
        (thetas[4],sub_values[4]),
        (thetas[5],sub_values[5])   
    ])
    result = sp.Matrix([
        [msubs[0,3]],
        [msubs[1,3]],
        [msubs[2,3]],
        [0],
        [0],
        [0]
    ])
    return result


In [62]:


#Validate transformation matrix:

val_matrix = sp.Matrix([
    [0, q, 0, 0, 0],
    [0, 0, q, 0, 0],
    [0, 0, 0, q, 0],
    [0, 0, 0, 0, q],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])

A_vals = [sp.zeros(4,4)]*5
for i in range(0,5):
    A_vals[i] = subs_function(A_final,val_matrix[:,i])


pos_vals = ["start position","first joint rotated 90 degrees","shoulder rotated 90 degrees","elbow rotated 90 degrees","wrist rotated 90 degrees"]
print("Transformation matrix validation")
for i in range(0,5):
    print("---------------")
    print(pos_vals[i])
    print(
        "x = ",((A_vals[i])[0,3]).round(2),
        "y = ",((A_vals[i])[1,3]).round(2),
        "z = ",((A_vals[i])[2,3]).round(2))




Transformation matrix validation
---------------
start position
x =  0 y =  -172.30 z =  1423.91
---------------
first joint rotated 90 degrees
x =  172.30 y =  0 z =  1423.91
---------------
shoulder rotated 90 degrees
x =  -1240.61 y =  -172.30 z =  183.30
---------------
elbow rotated 90 degrees
x =  -503.30 y =  -172.30 z =  920.61
---------------
wrist rotated 90 degrees
x =  -115.50 y =  -172.30 z =  1308.41


In [63]:
thetas_vector = sp.Matrix([thetas[0],thetas[1],thetas[2],thetas[3],thetas[4],thetas[5]])

P = sp.Matrix([A_final[0,3],A_final[1,3],A_final[2,3]])

Jv = P.jacobian(thetas_vector)
Z = sp.zeros(3,6)

for c in range(0,6):
    for r in range(0,3):
        Z[r,c] = (A_products[c])[r,2]


J = sp.zeros(6,6)
for c in range(0,6):
    for r in range(0,3):
        J[r,c] = Jv[r,c]
        J[r+3,c] = Z[r,c]




In [64]:
#initialize inputs
increment = 0.1 #time increment [s]
duration = 5 #total time [s]
arraysize = int(duration/increment) 
time = np.arange(0, duration, increment) #time array

#initial end effector posiitons with respect to base frame [mm]
ee_x_initial = 0 
ee_y_initial = -172.3
ee_z_initial = d1+d2+d3+d5

#Separate end effector path into 6 paths
path1_x_final = 0 #start position x of drawn shape
path1_y_final = -172.3 #start position y of drawn shape
path1_z_final = ee_z_initial-50 #start position z of drawn shape

#calculate distances traveled during paths


#calculate time taken for each path


#calculate start time of each path





In [65]:

#calculate path1 velocities from path 1 time and distance
velocity_matrix = sp.Matrix([0,0,-50/5,0,0,0])




In [66]:
from sympy import pprint



In [67]:
#calculate initial inverse jacobian to obtain initial thetadot values
almost_zeros = sp.Matrix([0.000001,0.000001,0.000001,0.000001,0.000001,0.000001])

jinv_initial = (subs_function(J,almost_zeros)).inv()
thetadot_initial = jinv_initial*velocity_matrix
thetadot_initial

# thetadot_initial = jinv_initial*XDOT[:,0] #initial thetadot values

# thetadot_initial













Matrix([
[    -2541.34772752892],
[    -8472.91695116742],
[     11014.2646786963],
[-1.65436122510606e-23],
[    -58987.3941740761],
[     58987.3941740761]])

In [68]:
#PATH1
#initialize theta/thetadot arrays with zeros
THETA_path1 = sp.zeros(6,arraysize)
THETADOT_path1 = sp.zeros(6,arraysize)


THETA_path1[:,0] = almost_zeros
THETADOT_path1[:,0] = thetadot_initial #populate thetadot with initial values

for r in range(0,6):
    THETA_path1[r,1] = (THETA_path1[r,1-1]+((THETADOT_path1[r,1-1])*increment))
THETA_path1
JSUBS = subs_function(J,THETA_path1.col(1))
jinv = JSUBS.inv()
thetadot = jinv*velocity_matrix
thetadot
for i in range(0,6):
    THETADOT_path1[i,1] = thetadot[i]
THETADOT_path1


#JSUBS = subs_function(J,THETA_path1.col(1))

# for c in range(1,arraysize):
#     for r in range(0,6):
#         THETA_path1[r,c] = (THETA_path1[r,c-1]+((THETADOT_path1[r,c-1])*increment))

#     JSUBS = subs_function(J,THETA_path1.col(c))
    
#     jinv = JSUBS.inv()
#     thetadot = jinv*velocity_matrix
    
#     for i in range(0,6):
#         THETADOT_path1[r,c] = thetadot[r]




Matrix([
[    -2541.34772752892, -0.00805258873042435, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[    -8472.91695116742,   0.0121256731140964, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[     11014.2646786963, -0.00407308438367209, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-1.65436122510606e-23, 4.33680868994202e-18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[    -58987.3941740761,  -0.0940712395893887, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[     58987.3941740761,   0.09407123

In [69]:
#PLOTTING END EFFECTOR POSITION
end_effector_xyz = sp.zeros(3,arraysize)
#PATH 1
for i in range(0,path1_arraysize):
    Asubs = A_final.subs([
        (thetas[0],THETA_path1[0,i]),
        (thetas[1],THETA_path1[1,i]),
        (thetas[2],THETA_path1[2,i]),
        (thetas[3],THETA_path1[3,i]),
        (thetas[4],THETA_path1[4,i]),
        (thetas[5],THETA_path1[5,i])])
    end_effector_xyz[0,i] = Asubs[0,3]
    end_effector_xyz[1,i] = Asubs[1,3]-45
    end_effector_xyz[2,i] = Asubs[2,3]


#PLOT END EFFECTOR PATH
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(0,path1_arraysize):
    
    ax.scatter([end_effector_xyz[0,i]], [end_effector_xyz[1,i]], [end_effector_xyz[2,i]])


# Set labels
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')


# Show plot
plt.show()

end_effector_xyz


NameError: name 'path1_arraysize' is not defined